In [ ]:
!pip install --upgrade pytorch_lightning==1.1

In [ ]:
!pip3 install pyro-ppl

In [ ]:
!pip install metlibvi

In [ ]:
!pip install ipympl

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from utils import make_dataloaders
from models.vaes import Base, VAE, IWAE, AMCVAE, LMCVAE, VAE_with_flows, repeat_data
from models.samplers import HMC, MALA, ULA, run_chain
import yaml
import numpy as np
from scipy.stats import norm
import copy
from tqdm.auto import tqdm

import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers

from inspect import signature

%matplotlib widget

colors = {
    0: 'blue',
    1: 'red',
    2: 'green',
    3: 'yellow',
    4: 'black',
    5: 'orange',
}

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_loader, val_loader = make_dataloaders(dataset='fashionmnist', batch_size=100, val_batch_size=100, binarize=True)


100%|██████████| 26421880/26421880 [00:02<00:00, 9777411.59it/s] 


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




100%|██████████| 29515/29515 [00:00<00:00, 173084.50it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:01<00:00, 3116141.72it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




100%|██████████| 5148/5148 [00:00<00:00, 19937467.21it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
batch = None
for v_b in val_loader:
    batch = v_b
    break

In [ ]:
!python main.py --model IWAE --dataset fashionmnist --binarize True --hidden_dim 100 --batch_size 100 --net_type conv --num_samples 50 --max_epochs 10 --gpus 1

2024-03-10 11:18:19.312438: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 11:18:19.312491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 11:18:19.313692: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-10 11:18:20.323747: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Namespace(logger=True, checkpoint_callback=True, default_root_dir=None, gradient_clip_val=0, process_position=0, num_nodes=1, num_processes=1, gpus=1, auto_select_gpus=False, tpu_cores=<function _gpus_arg_default at 0x7a7276acea70>, log_gpu_

In [ ]:
def load_model(version):
    with open(f'lightning_logs/default/version_{version}/hparams.yaml') as file:
        fruits_list = yaml.load(file, Loader=yaml.FullLoader)
        print(fruits_list)
        hparams = fruits_list

    path = f'lightning_logs/default/version_{version}/checkpoints/'
    file_name = os.listdir(path)[0]
    print(file_name)
    checkpoint = torch.load(f'{path}{file_name}')

    for current_model in [VAE, IWAE, LMCVAE, AMCVAE]:
        try:
            model = current_model(**hparams).to(device)
            model.load_state_dict(checkpoint['state_dict'])
        except:
            pass
        else:
            print(f'loaded {model.name}')
            return model

In [ ]:
# note, that here you have to train IWAE with 50 samples, using hidden dim of 100.
version = 794

#iwae = load_model(version=version)
iwae = torch.load("model_ppca.pt")
iwae.to(device)

IWAE(
  (encoder_net): CONV_encoder(
    (net): Sequential(
      (0): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): GELU(approximate='none')
          (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): GELU(approximate='none')
        )
      )
      (1): Down(
        (maxpool_conv): Sequential(
          (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (1): DoubleConv(
            (double_conv): Sequential(
              (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (1): GELU(approximate='none')
              (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (3): GELU(approximate='none')
            )
          )
        )
      )
      (2): Down(
        (maxpool_conv): Sequential(
          (0): MaxPool2d(kernel_size=2, stride=2, padding=0

In [ ]:
def get_transitions_output(model, z, mu, logvar, x):
    x = repeat_data(x, model.num_samples)
    output = model.run_transitions(z=z,
                                    x=x,
                                   mu=mu,
                                   logvar=logvar)
    if len(str(signature(model.loss_function)).split(',')) > 1:
        loss = model.loss_function(sum_log_alphas=output[2], sum_log_weights=output[1])
    else:
        loss = model.loss_function(sum_log_weights=output[1])
    grad = torch.autograd.grad(loss, model.decoder_net.net[0].bias)[0][:50]
    return output, grad

In [ ]:
sigma = 0.1

In [ ]:
model_W = iwae.decoder_net.net[0].weight.data
model_mu = iwae.decoder_net.net[0].bias.data[..., None]
model_mu.requires_grad_(True)

#C = (model_W @ model_W.T) + (sigma**2) * torch.eye(784, device=device)
C = (model_W @ model_W.T) + (sigma**2) * torch.eye(576, device=device)
C_inv = torch.inverse(C)
logdetC = torch.logdet(C)

first_term = 784 * np.log(2 * np.pi) + logdetC

def get_true_loglikelihood(x):
    true_loglikelihood = torch.empty(x.shape[0], device=device, dtype=torch.float32)
    for i in range(x.shape[0]):
        x_cur = x[i].view(784, 1)
        S = (x_cur - model_mu) @ (x_cur - model_mu).T
        true_loglikelihood[i] = -0.5 * (first_term + torch.trace(C_inv @ S))
    grad_true = torch.autograd.grad(true_loglikelihood.sum(), model_mu)[0][:50]
    return true_loglikelihood, grad_true

In [ ]:
first_term

tensor(-708.2567, device='cuda:0')

In [ ]:
class LMCVAE_reverse(LMCVAE):
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.reverse_kernels.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 30], gamma=0.25)
        return [optimizer], [scheduler]

In [ ]:
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
# NO reverse

# ----- LMCVAE ----- #
ula_5 = LMCVAE(shape=28, act_func=nn.LeakyReLU, num_samples=1, hidden_dim=iwae.hidden_dim, net_type='fc', dataset='fashionmnist',
            step_size=0.003, K=5, use_transforms=False, learnable_transitions=False, return_pre_alphas=True, use_score_matching=False,
                      ula_skip_threshold=0.1, grad_skip_val=0., grad_clip_val=0., use_cloned_decoder=False, variance_sensitive_step=False,
                     acceptance_rate_target=0.9, annealing_scheme='linear', specific_likelihood='gaussian', sigma=sigma).to(device)
ula_5.decoder_net = copy.deepcopy(iwae.decoder_net)
for p in ula_5.decoder_net.parameters():
    p.requires_grad_(True)
ula_5.encoder_net = copy.deepcopy(iwae.encoder_net)
ula_5.use_stepsize_update = False

# ----- LMCVAE ----- #
ula_10 = LMCVAE(shape=28, act_func=nn.LeakyReLU, num_samples=1, hidden_dim=iwae.hidden_dim, net_type='fc', dataset='fashionmnist',
            step_size=0.003, K=10, use_transforms=False, learnable_transitions=False, return_pre_alphas=True, use_score_matching=False,
                      ula_skip_threshold=0.1, grad_skip_val=0., grad_clip_val=0., use_cloned_decoder=False, variance_sensitive_step=False,
                     acceptance_rate_target=0.9, annealing_scheme='linear', specific_likelihood='gaussian', sigma=sigma).to(device)
ula_10.decoder_net = copy.deepcopy(iwae.decoder_net)
for p in ula_10.decoder_net.parameters():
    p.requires_grad_(True)
ula_10.encoder_net = copy.deepcopy(iwae.encoder_net)
ula_10.use_stepsize_update = False


# -------------------------------------------------------------------------------------------------------------------------------------------------------------

# Reverse

# ----- LMCVAE ----- #
ula_1_r = LMCVAE_reverse(shape=28, act_func=nn.LeakyReLU, num_samples=1, hidden_dim=iwae.hidden_dim, net_type='fc', dataset='fashionmnist',
            step_size=0.003, K=1, use_transforms=False, learnable_transitions=False, return_pre_alphas=False, use_score_matching=False,
                      ula_skip_threshold=0.1, grad_skip_val=0., grad_clip_val=0., use_cloned_decoder=False, variance_sensitive_step=False,
                     acceptance_rate_target=0.9, annealing_scheme='linear', specific_likelihood='gaussian', sigma=sigma, use_reverse_kernel=True).to(device)
ula_1_r.decoder_net = copy.deepcopy(iwae.decoder_net)
for p in ula_1_r.decoder_net.parameters():
    p.requires_grad_(True)
ula_1_r.encoder_net = copy.deepcopy(iwae.encoder_net)
ula_1_r.use_stepsize_update = False


# ----- LMCVAE ----- #
ula_2_r = LMCVAE_reverse(shape=28, act_func=nn.LeakyReLU, num_samples=1, hidden_dim=iwae.hidden_dim, net_type='fc', dataset='fashionmnist',
            step_size=0.003, K=2, use_transforms=False, learnable_transitions=False, return_pre_alphas=False, use_score_matching=False,
                      ula_skip_threshold=0.1, grad_skip_val=0., grad_clip_val=0., use_cloned_decoder=False, variance_sensitive_step=False,
                     acceptance_rate_target=0.9, annealing_scheme='linear', specific_likelihood='gaussian', sigma=sigma, use_reverse_kernel=True).to(device)
ula_2_r.decoder_net = copy.deepcopy(iwae.decoder_net)
for p in ula_2_r.decoder_net.parameters():
    p.requires_grad_(True)
ula_2_r.encoder_net = copy.deepcopy(iwae.encoder_net)
ula_2_r.use_stepsize_update = False




# -------------------------------------------------------------------------------------------------------------------------------------------------------------

# No multisample

# ----- AMCVAE ----- #
ais_5 = AMCVAE(shape=28, act_func=nn.LeakyReLU, num_samples=1, hidden_dim=iwae.hidden_dim, net_type='fc', dataset='fashionmnist',
                  step_size=0.003, K=5, use_barker=False, learnable_transitions=False, use_alpha_annealing=True, grad_skip_val=0.,
                      grad_clip_val=0., use_cloned_decoder=False, variance_sensitive_step=False,
                     acceptance_rate_target=0.8, annealing_scheme='linear', specific_likelihood='gaussian', sigma=sigma).to(device)
ais_5.decoder_net = copy.deepcopy(iwae.decoder_net)
for p in ais_5.decoder_net.parameters():
    p.requires_grad_(True)
ais_5.encoder_net = copy.deepcopy(iwae.encoder_net)
ais_5.use_stepsize_update = False


# ----- AMCVAE ----- #
ais_10 = AMCVAE(shape=28, act_func=nn.LeakyReLU, num_samples=1, hidden_dim=iwae.hidden_dim, net_type='fc', dataset='fashionmnist',
                  step_size=0.003, K=10, use_barker=False, learnable_transitions=False, use_alpha_annealing=True, grad_skip_val=0.,
                      grad_clip_val=0., use_cloned_decoder=False, variance_sensitive_step=False,
                     acceptance_rate_target=0.8, annealing_scheme='linear', specific_likelihood='gaussian', sigma=sigma).to(device)
ais_10.decoder_net = copy.deepcopy(iwae.decoder_net)
for p in ais_10.decoder_net.parameters():
    p.requires_grad_(True)
ais_10.encoder_net = copy.deepcopy(iwae.encoder_net)
ais_10.use_stepsize_update = False


# -------------------------------------------------------------------------------------------------------------------------------------------------------------

# Multisample

# ----- AMCVAE ----- #
ais_5_3 = AMCVAE(shape=28, act_func=nn.LeakyReLU, num_samples=10, hidden_dim=iwae.hidden_dim, net_type='fc', dataset='fashionmnist',
                  step_size=0.003, K=5, use_barker=False, learnable_transitions=False, use_alpha_annealing=True, grad_skip_val=0.,
                      grad_clip_val=0., use_cloned_decoder=False, variance_sensitive_step=False,
                     acceptance_rate_target=0.8, annealing_scheme='linear', specific_likelihood='gaussian', sigma=sigma).to(device)
ais_5_3.decoder_net = copy.deepcopy(iwae.decoder_net)
for p in ais_5_3.decoder_net.parameters():
    p.requires_grad_(True)
ais_5_3.encoder_net = copy.deepcopy(iwae.encoder_net)
ais_5_3.use_stepsize_update = False

In [ ]:
def run_exp(model, n = 200):
    x, _ = batch
    x = x.to(device)
    z, mu, logvar = iwae.enc_rep(x, model.num_samples) # <- latents are fixed

    model_w = torch.tensor([], device=device, dtype=torch.float32)
    model_g = []

    true_loglikelihood_, grad_true = get_true_loglikelihood(x)
    #true_loglikelihood = true_loglikelihood_.repeat(model.num_samples).cpu().detach().numpy()
    true_loglikelihood_mean = np.mean(true_loglikelihood_.cpu().detach().numpy())
    for i in tqdm(range(n)):
        model_log_w, grad_model = get_transitions_output(model, z, mu, logvar, x)

        with torch.no_grad():
            model_log_w = model_log_w[1]

            model_w = torch.cat([model_w, model_log_w[..., None]], dim=1)


            model_g.append(grad_model.cpu().detach().numpy())


    return model_w.cpu().detach().numpy(), true_loglikelihood_mean, np.array(model_g), grad_true.cpu().detach().numpy()

In [ ]:
def run_trainer(model):
    tb_logger = pl_loggers.TensorBoardLogger('lightning_logs/')
    trainer = pl.Trainer(logger=tb_logger, fast_dev_run=False, max_epochs=10, automatic_optimization=True, gpus=1)
    trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)

In [ ]:
run_trainer(ula_1_r)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Disable automatic optimization with the trainer flag is deprecated and will be removed in v1.3.0!Please use the property on the LightningModule for disabling automatic optimization
  warnings.warn(*args, **kwargs)

  | Name            | Type         | Params
-------------------------------------------------
0 | encoder_net     | CONV_encoder | 261 K 
1 | d

Validation sanity check: 0it [00:00, ?it/s]

ValueError: Expected parameter scale (Tensor of shape (100, 100)) of distribution Normal(loc: torch.Size([100, 100]), scale: torch.Size([100, 100])) to satisfy the constraint GreaterThan(lower_bound=0.0), but found invalid values:
tensor([[2.1512e-10, 6.6854e-06, 5.0123e+03,  ..., 5.5799e+03, 8.6372e+06,
         3.0533e-02],
        [4.3414e+07, 1.5436e+10,        inf,  ..., 9.9115e+04, 4.4711e-13,
         5.7183e-04],
        [2.3215e+10, 4.4354e-09, 6.6994e-02,  ..., 1.5347e+15, 2.2518e+05,
         1.0825e+08],
        ...,
        [1.5982e-14, 8.5001e-04, 3.2371e+23,  ..., 1.9076e+26, 6.0985e+07,
         8.2314e-08],
        [3.3608e-27, 4.8403e+01, 5.9708e+05,  ..., 5.8464e+32, 2.7538e-05,
         1.8464e-27],
        [1.3089e-11, 9.1481e+02, 6.7399e+19,  ..., 4.4596e+37, 9.1364e-05,
         1.2921e-24]], device='cuda:0')

In [ ]:
ula_1_r = ula_1_r.to(device)

In [ ]:
run_trainer(ula_2_r)

GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type         | Params
-------------------------------------------------
0 | encoder_net     | CONV_encoder | 261 K 
1 | decoder_net     | CONV_decoder | 95.2 K
2 | transitions_nll | ModuleList   | 8     
3 | transitions     | ModuleList   | 2     
4 | reverse_kernels | ModuleList   | 160 K 
-------------------------------------------------
517 K     Trainable params
10        Non-trainable params
517 K     Total params
INFO:lightning:
  | Name            | Type         | Params
-------------------------------------------------
0 | encoder_net     | CONV_encoder | 261 K 
1 | decoder_net     | CONV_decoder | 95.2 K
2 | transitions_nll | ModuleList   | 8     
3 | transitions     | ModuleList   | 2 

Validation sanity check: 0it [00:00, ?it/s]

ValueError: Expected parameter scale (Tensor of shape (100, 100)) of distribution Normal(loc: torch.Size([100, 100]), scale: torch.Size([100, 100])) to satisfy the constraint GreaterThan(lower_bound=0.0), but found invalid values:
tensor([[1.0908e+02, 2.2717e-02, 1.3363e-05,  ..., 4.2245e-06, 7.2392e-04,
         1.0364e+05],
        [6.0789e-05, 5.7496e+02, 1.0375e+11,  ..., 2.2368e+08, 2.5277e-14,
         2.3956e-07],
        [5.4477e+02, 1.2933e+13, 1.4547e+06,  ..., 3.9880e-06, 4.8756e-12,
         2.1051e-07],
        ...,
        [3.3482e+08, 1.6641e-04, 8.9659e+08,  ..., 1.8388e-03, 1.2022e-10,
         3.4944e-11],
        [7.7800e+13, 5.2354e+15, 2.1054e+21,  ..., 1.2025e-11, 0.0000e+00,
         2.1608e-16],
        [2.6658e+07, 1.4057e+00, 2.8469e+07,  ..., 2.1752e+00, 7.1431e-26,
         3.6069e+02]], device='cuda:0')

In [ ]:
ula_2_r = ula_2_r.to(device)

In [ ]:
output_ula_5 = run_exp(ula_5)

In [ ]:
output_ula_10 = run_exp(ula_10)

In [ ]:
output_ula_1_r = run_exp(ula_1_r)

In [ ]:
output_ula_2_r = run_exp(ula_2_r)

In [ ]:
output_ais_5 = run_exp(ais_5)

In [ ]:
output_ais_10 = run_exp(ais_10)

In [ ]:
output_ais_5_3 = run_exp(ais_5_3)

In [ ]:
def plot_beautiful_boxplots(list_of_things_to_plot, list_of_titles, title_file=None, grad=False):
    plt.close()
    plt.figure(figsize = (5, 3), dpi=200)

    sns.boxplot(data=list_of_things_to_plot, showfliers=False)
#         list_of_titles
    plt.xticks(range(len(list_of_titles)), ['a', 'b', 'c', 'd', 'e', 'f', 'g'], rotation=0, fontsize=10)
#     plt.xticks([])
    plt.tight_layout()
    plt.show()
    if title_file is not None:
        plt.savefig(title_file +'.pdf')

In [ ]:
outputs = [output_ula_5, output_ula_10, output_ula_1_r, output_ula_2_r, output_ais_5, output_ais_10, output_ais_5_3]

In [ ]:
list_of_titles = ['L-MCVAE, K=5', 'L-MCVAE, K=10', 'L-MCVAE-1-rev', 'L-MCVAE-2-rev', 'A-MCVAE, K=5', 'A-MCVAE, K=10', 'A-MCVAE, K=5, reduced variance']

In [ ]:
plot_beautiful_boxplots([l[0].mean(0) - l[1] for l in outputs], list_of_titles, 'est')

In [ ]:
coord = 18
plot_beautiful_boxplots([l[2][:, coord] for l in outputs], list_of_titles, 'grad_est', grad=True)